In [1]:
import os
import re
import json
import glob
import string
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

## Read & Merge Datasets

### LinkedIn Job Postings

In [2]:
df_li = pd.read_csv("./Kaggle-Data-Science-ML-Survey-Analysis/data/raw/job_listings/MLE Jobs LinkedIn.csv")
df_li.head(3)

,Unnamed: 0,Job Title,Company Name,Location,Work Mode,Employment Type,Company Headcount,Industry,JD,JobID
0,0,SENIOR COMPUTER VISION DATA SCIENTIST,80 Acres Farms,"Hamilton, OH",On-site,Full-time · Mid-Senior level,201-500 employees,Farming,About The Company\r\n\r\nThe Future of Fresh i...,3331975665
1,1,Senior Software Engineer,Terran Robotics,"Bloomington, IN",Hybrid,Full-time,1-10 employees,None,Terran Robotics is building construction robot...,2841811846
2,2,Systems Engineer,ZF Group,Greater Lansing,Hybrid,Full-time · Mid-Senior level,"10,001+ employees",Motor Vehicle Manufacturing,Job description:\r\n\r\nJob Description:\r\n\r...,3369017400


### Glassdoor Job Postings

In [3]:
path_folder = "./Kaggle-Data-Science-ML-Survey-Analysis/data/raw/job_listings"
paths_csv = glob.glob(os.path.join(path_folder, "*.csv"))

cols = ['Job Title', 'Salary Estimate', 'Job Description', 'Rating', 'Company Name', 'Location', 'Headquarters',
         'Size', 'Founded', 'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors', 'Easy Apply']

df_gd = pd.DataFrame()

for file in paths_csv:
    if "MLE Jobs LinkedIn" in file:
        continue
    jobfamily = file.split("/")[-1].split(".csv")[0] # Extract job family from file path    
    df = pd.read_csv(file)
    df = df[cols] # Reorder & subset cols for data integrity
    df['Job Family']= jobfamily
    df_gd = pd.concat([df_gd, df], axis=0, ignore_index=True)

del df
df_gd.head(3)

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply,Job Family
0,Senior Data Scientist,$111K-$181K (Glassdoor est.),"ABOUT HOPPER\n\nAt Hopper, we’re on a mission ...",3.5,Hopper\n3.5,"New York, NY","Montreal, Canada",501 to 1000 employees,2007,Company - Private,Travel Agencies,Travel & Tourism,Unknown / Non-Applicable,-1,-1,DataScientist
1,"Data Scientist, Product Analytics",$111K-$181K (Glassdoor est.),"At Noom, we use scientifically proven methods ...",4.5,Noom US\n4.5,"New York, NY","New York, NY",1001 to 5000 employees,2008,Company - Private,"Health, Beauty, & Fitness",Consumer Services,Unknown / Non-Applicable,-1,-1,DataScientist
2,Data Science Manager,$111K-$181K (Glassdoor est.),Decode_M\n\nhttps://www.decode-m.com/\n\nData ...,-1.0,Decode_M,"New York, NY","New York, NY",1 to 50 employees,-1,Unknown,-1,-1,Unknown / Non-Applicable,-1,True,DataScientist


### Merge Datasets

In [4]:
cols = ["Job Title", "Job Description"]

df_li = df_li.rename(columns={"JD": "Job Description"})
df = pd.concat([df_li[cols], df_gd[cols]], axis=0, ignore_index=True)

## Preprocess Job Titles

In [5]:
df = df.dropna(subset=['Job Title'])

regex_punct = re.compile('[%s]' % re.escape(string.punctuation))

def remove_punct(text):
    cleaned = regex_punct.sub(' ', text) # Replace punctuations with spaces. Exp: "AI/ML" -> "AI ML"
    return re.sub(' +', ' ', cleaned) # Replace multiple spaces with single space

In [6]:
df['cleaned_job_title'] = df['Job Title'].apply(remove_punct)

In [7]:
jobs = np.sort(df['Job Title'].dropna().unique())
jobs

array(['!!!100% Remote!!! Sr. Data and Integration Engineer',
       '"Business Analyst/ Data Analyst with experience on any one of Financial Product',
       '#104252 Division Data and Financial Analyst', ...,
       'systems Analyst 1596 (d)',
       '{"title":"GAO Analyst (Data Analysis)","extended":{"opening_type":null}}',
       '【1yr OPT+Intern】Data Analyst 保实习保就业'], dtype=object)

# Method 1

## Keyword Selection (Iterative Cycle)
Define KWs, test matches & iterate

### Define Keywords

In [8]:
job_mapping = {
    
    "Business Analyst": ["business analyst", "business analytics", "business intelligence", "bi analyst",
                        "bi engineer"],
    
    "Data Analyst": ["data analyst", "data analytics", "analytics engineer", "analytics consult"
                    "analytics manager", "decision analyst"],
    
    "Data Engineer": ["big data engineer", "data engineer", "data science engineer", "data framework engineer",
                      "data visualization engineer",
                     "kafka"], #"aws", "azure"
    
    "Data Scientist": ["data scientist", "decision scientist"],
    
    "Machine Learning Engineer": ["machine learning", "ml engineer", "applied scientist", 
                        "artificial intelligence", "deep learning", "ai engineer", "ai scientist", "ai ml",
                        "computer vision"],
    
    "Research": ["research"]
    
}

### Test Matches

In [9]:
def match_kw(jd, kws):
    "Returns 1 if the JD contains a keyword associated to a skill"
    for kw in kws:
        if kw.lower() in jd.lower():
            return 1
    return 0

In [10]:
mappings = {title:[] for title in job_mapping.keys()}
mappings["other"] = []

jobs = np.sort(df['cleaned_job_title'].dropna().unique())

for job in jobs:
    anymatch = 0
    
    for title, kws in job_mapping.items():
        matched = match_kw(job, kws)
        if matched:
            mappings[title].append(job)
            anymatch = 1
    
    if anymatch == 0:
        mappings["other"].append(job)

In [11]:
# Get counts of each category
{title: len(mappings[title]) for title in mappings.keys()}

{'Business Analyst': 844,
 'Data Analyst': 1730,
 'Data Engineer': 443,
 'Data Scientist': 444,
 'Machine Learning Engineer': 245,
 'Research': 270,
 'other': 3384}

In [12]:
# Interesting to see the large number of ways of describing each job title!
# Some confusing job titles like: Data Science Analytics Engineer

### Shortcomings:
1. The 'Other' category contains the highest number of job titles.
2. If we rely on simple keyword based matching, we have to manually define a large number of keywords for each job category.

In [13]:
idxs = [19, 40, 90, -55]   # Selected exps for illustration
for idx in idxs:
    print(mappings['other'][idx])

077 Data Science Analyst
AVP Business Systems Analyst
Analyst Data
VP Data Science


# Method 2

## Fuzzy String matching

### Strategy
1. Find fuzzy score of job title with each defined title. Exp (Agile Data Product Analyst, Data Analyst)
2. If the highest fuzzy score is greater than a predefined threshold, add the title to the category.
3. If the fuzzy score is lower, mark the title as 'Other'.
4. Find a 'good' threshold based on experimentation.

In [14]:
from fuzzywuzzy import fuzz

In [15]:
mappings_fuzzy = {title:[] for title in job_mapping.keys()}
mappings_fuzzy["other"] = []

thresh = 30
titles = ['Business Analyst', 'Data Analyst', 'Data Engineer', 'Data Scientist', 
                  'Machine Learning Engineer', 'Research']

jobs = np.sort(df['cleaned_job_title'].dropna().unique())

for job in jobs:
    
    fuzz_scores = [fuzz.ratio(job.lower(), title.lower()) for title in titles] 
    # Note: Case is taken into account for fuzzy matching
    
    max_score, max_score_idx = np.max(fuzz_scores), np.argmax(fuzz_scores)
    
    if max_score >= thresh:
        mappings_fuzzy[titles[max_score_idx]].append(job)
    
    else:
        mappings_fuzzy["other"].append(job)

In [16]:
# Get counts of each category
{title: len(mappings_fuzzy[title]) for title in mappings.keys()}

{'Business Analyst': 1713,
 'Data Analyst': 2218,
 'Data Engineer': 1137,
 'Data Scientist': 1104,
 'Machine Learning Engineer': 885,
 'Research': 11,
 'other': 96}

### Shortcomings:
1. When the job title is long, Fuzzy matching fails even if the exact term is present. Therefore, the 'Other' category mostly contains long titles.
2. If we reduce the threshold, we get very noisy labels.
3. Some obvious misses: Data Quality and Process Analyst -> Business Analyst, Cybersecurity Analyst -> Business Analyst.

In [17]:
idxs = [7, 27, 32]   # Selected exps for illustration ("Other" category)
for idx in idxs:
    print(mappings_fuzzy['other'][idx])

Business Analyst Advisory Services Performance Improvement Finance Transformation Manager Multiple Positions 
Data Analyst Data Modeler ETL Developer Data Management 162020CIT ITS NEASO Internal Only Posti
Data Scientist 2 Tableau Statistics IT Portfolio Predictive Analysis Visualization 


# Method 3

### Multi-word filtering

For example, consider the job titles:
* Business Process Support Analyst
* Business Reporting Analyst II
* Business Support Analyst Senior

These can be filtered by checking for the presence of both 'business' and 'analyst'

In [18]:
job_mapping_multiword = {
    
    "Business Analyst": ["business_analyst"],
    
    "Data Analyst": ["data_analyst", "data_analytics", "decision_analyst"],
    
    "Data Engineer": ['data_engineer'],
    
    "Data Scientist": ["data_scientist"],
    
    "Machine Learning Engineer": ["ai_engineer"],
}

In [19]:
def match_kw_list(jd, multi_kw_set):
    """
    Returns 1 if the JD contains all keywords associated to a skill.
    For exp, 'Business Reporting Analyst II' matches 'Business Analyst', since it contains both words
    'business' and 'analyst'.
    """
    kws_list = multi_kw_set.split("_")
    matches = [kw.lower() in jd.lower() for kw in kws_list]
    if np.sum(matches) == len(matches):
            return 1
    return 0

In [20]:
mappings_multiword = {title:[] for title in job_mapping.keys()}
mappings_multiword["other"] = []

jobs = mappings['other'] # Tesing on the 'Other' category of simple keyword matching

for job in jobs:
    anymatch = 0
    
    for title, kws_list in job_mapping_multiword.items():
        for multi_kw_set in kws_list:
            matched = match_kw_list(job, multi_kw_set)
            if matched:
                mappings_multiword[title].append(job)
                anymatch = 1
    
    if anymatch == 0:
        mappings_multiword["other"].append(job)

In [21]:
# Get counts of each category
{title: len(mappings_multiword[title]) for title in mappings.keys()}

{'Business Analyst': 287,
 'Data Analyst': 760,
 'Data Engineer': 176,
 'Data Scientist': 8,
 'Machine Learning Engineer': 22,
 'Research': 0,
 'other': 2178}

In [22]:
mappings_multiword['other'][10:20]

['3rd Shift Environmental Monitoring Scientist',
 'ACH Analyst',
 'ADAS Test V V Engineer',
 'AIG Retirement Services Plan Administration Analyst I',
 'AM PM Building Engineer Position',
 'AML Compliance Analyst',
 'AMS Verification and Modeling Engineer',
 'ANALYST ENGINEER SCIENTIST ACOUSTICS',
 'ANALYST SCIENTIST KINETICS',
 'APPLICATION SUPPORT ANALYST I']

### Comments
1. Many titles in the 'Other' category look ambiguous. For exp: Data Specialist, Data Lead, VP Data Science, Data Science with strong Python, Risk Analyst. So this method does a good job overall.
2. However, some obvious titles are still missed. For exp: Business Analysis, Ecommerce Analytics Manager. These can be matched with Fuzzy search. 

# Method 4

### Simple Keyword matching > Multi-keyword Matching > Fuzzy matching (strict)

In [23]:
# Finding a reasonable threshold for fuzzy matching on the other category of multiword mathcing

In [24]:
mappings_fuzzy = {title:[] for title in job_mapping.keys()}
mappings_fuzzy["other"] = []

thresh = 80
titles = ['Business Analyst', 'Data Analyst', 'Data Engineer', 'Data Scientist', 
                  'Machine Learning Engineer', 'Research']

jobs = mappings_multiword['other'] # Other category of multiword matching

for job in jobs:
    
    fuzz_scores = [fuzz.ratio(job.lower(), title.lower()) for title in titles] 
    # Note: Case is taken into account for fuzzy matching
    
    max_score, max_score_idx = np.max(fuzz_scores), np.argmax(fuzz_scores)
    
    if max_score >= thresh:
        mappings_fuzzy[titles[max_score_idx]].append(job)
    
    else:
        mappings_fuzzy["other"].append(job)

In [25]:
# Get counts of each category
{title: len(mappings_fuzzy[title]) for title in mappings.keys()}

{'Business Analyst': 2,
 'Data Analyst': 3,
 'Data Engineer': 5,
 'Data Scientist': 2,
 'Machine Learning Engineer': 1,
 'Research': 0,
 'other': 2165}

In [26]:
mappings_fuzzy['Business Analyst']

['Business Analysis', 'IT Business Analysis']

In [27]:
mappings_fuzzy['Data Analyst']

['Digital Analyst', 'IT Analyst', 'QA Analyst']

In [28]:
mappings_fuzzy['Data Engineer']

[' Java Engineer',
 'AWS Engineer',
 'IAM Engineer',
 'Java Engineer',
 'QA Engineer']

In [29]:
mappings_fuzzy['Data Scientist']

['Data Sceintist', 'Staff Scientist']

In [30]:
mappings_fuzzy['Machine Learning Engineer']

['Machine Learninig Engineer']

### Comments
1. With fuzzy matching, we are able to accomodate for spelling mistakes in job titles. For exp: "Data Sceintist", "Machine Learinig Engineer".
2. However, even with a high threshold, we see irrelevant additions such "QA Test Engineer", "Field Scientist", "Java Engineer" 
3. We use the output of fuzzy matching to add keywords to our title list. See the list below.

In [31]:
fuzzy_kws = {
    
    "Business Analyst": ['Business Analysis',
                         'Business Data Modeler',
                         'Business Data Steward',
                         'Director of Business Analysis',
                         'IT Business Analysis',
                         'IT Business Process Analysis',
                         'Manager Business Analysis',
                         'Manager of Business Analysis',
                         'Professional Business Analysis',
                         'Sr Bus Analyst'],
    
    "Data Analyst": ['Quantitative Analyst',
                     'SAS Analyst',
                     'Tableau Analyst'], # Analytical Scientist
    
    "Data Engineer": ["Sr Big data Ebgineer", "Data Developer"],
    
    "Data Scientist": ["Data Sceintist", 'Data Science Intern', 'Data Science Specialist', 
                       'Statistical Scientist', # Staff Scientist
                       'Data Science Practitioner'], # Data Science Architect, Lead, VP, Director, Consultant

    
    "Machine Learning Engineer": ["Machine Learninig Engineer"],
    
}

# Tying it together

In [32]:
job_mapping = {}  # {Job Family: [Job_titles]} 

for title in mappings.keys():
    if title != "other":
        job_mapping[title] = mappings[title] + mappings_multiword[title] + mappings_fuzzy[title]

In [33]:
# Export
with open("Kaggle-Data-Science-ML-Survey-Analysis/data/raw/job_listings/job_mapping.json", 'w') as f:
    json.dump(job_mapping, f, indent=2) 

# Future work
1. Record all keywords in a config file.
2. Named Entity Recognition for detection of Job Titles.